In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import pickle
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
path = "csv/mldata_v2.csv"
df = pd.read_csv(path)
df_new = df.drop(columns = ['filename','length'])
df_new

,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,spectral_flatness_mean,spectral_flatness_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,0.335555,0.090997,0.130189,0.003559,1773.358004,169450.829707,1972.334258,117272.640573,0.004887,0.000106,...,39.547077,-3.230046,36.606857,0.696385,37.766136,-5.035945,33.668550,-0.239585,43.818886,blues
1,0.343523,0.086782,0.112119,0.001491,1817.244034,90766.297514,2010.751494,65940.666037,0.004589,0.000030,...,64.819780,-6.025473,40.548813,0.127131,51.048943,-2.808956,97.221504,5.771881,60.360348,blues
2,0.347746,0.092495,0.130895,0.004552,1790.722358,110071.206762,2088.184750,73391.498088,0.003933,0.000014,...,68.306790,-1.714475,28.136944,2.329553,47.211426,-1.925621,52.922430,2.466996,33.163998,blues
3,0.363863,0.087207,0.131349,0.002338,1660.545231,109496.936309,1967.920582,79805.901501,0.003744,0.000011,...,48.543200,-3.786986,28.419546,1.153315,35.682700,-3.501979,50.610344,3.580636,32.325880,blues
4,0.335481,0.088482,0.142370,0.001734,1634.465076,77425.419156,1954.633566,57359.695597,0.003332,0.000010,...,30.829544,0.635798,44.645557,1.591107,51.415867,-3.364908,26.421090,0.501504,29.109533,blues
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9976,0.348781,0.080553,0.049680,0.000098,1503.238679,163419.048820,1725.514991,85476.555708,0.001901,0.000006,...,44.396152,-9.084439,38.929996,-4.216517,31.358320,-5.681930,47.543434,1.725083,38.689384,rock
9977,0.372556,0.082633,0.057615,0.000096,1852.706841,277236.230333,1914.481257,97378.034110,0.003548,0.000027,...,32.134663,-12.323830,68.098270,-3.130632,53.029488,-11.916323,63.618256,0.384525,18.866629,rock
9978,0.347815,0.089140,0.051906,0.000674,1348.383672,662359.246177,1566.961069,137898.246581,0.002681,0.000017,...,79.231080,-2.683274,22.830883,4.947066,25.907822,1.718840,47.727330,-0.214869,41.377610,rock
9979,0.386801,0.084649,0.065967,0.000312,2086.306423,200732.442344,2021.144027,20772.677089,0.004326,0.000010,...,27.924114,-5.355850,17.127592,6.417248,21.935257,2.347247,24.930906,0.666209,12.577226,rock


In [3]:
y = df_new["label"].values
X = df_new.drop(columns='label').values

X_train, X_test, y_train, y_test = train_test_split(X, y)
scaler = StandardScaler()

X_scaler = scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [4]:
# DecisionTree model
model1 =  DecisionTreeClassifier()

model1.fit(X_train, y_train)

y_pred = model1.predict(X_test)


print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       blues       0.63      0.60      0.62       253
   classical       0.87      0.87      0.87       260
     country       0.52      0.57      0.55       249
       disco       0.58      0.62      0.60       250
      hiphop       0.66      0.66      0.66       235
        jazz       0.63      0.62      0.63       245
       metal       0.80      0.78      0.79       261
         pop       0.74      0.71      0.72       235
      reggae       0.66      0.67      0.67       258
        rock       0.52      0.48      0.50       250

    accuracy                           0.66      2496
   macro avg       0.66      0.66      0.66      2496
weighted avg       0.66      0.66      0.66      2496



In [5]:
# RandomForest model
model2 = RandomForestClassifier(n_estimators=100)
model2.fit(X_train, y_train)

y_pred = model2.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       blues       0.89      0.87      0.88       253
   classical       0.91      0.95      0.93       260
     country       0.82      0.89      0.86       249
       disco       0.81      0.86      0.84       250
      hiphop       0.92      0.85      0.88       235
        jazz       0.87      0.91      0.89       245
       metal       0.90      0.93      0.92       261
         pop       0.87      0.89      0.88       235
      reggae       0.89      0.87      0.88       258
        rock       0.86      0.72      0.78       250

    accuracy                           0.87      2496
   macro avg       0.87      0.87      0.87      2496
weighted avg       0.88      0.87      0.87      2496



In [6]:
# XGBoost model
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

label_encoder = LabelEncoder()

y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


model = xgb.XGBClassifier(n_estimators=100)
model.fit(X_train, y_train_encoded)

y_pred = model.predict(X_test)

y_pred_decoded = label_encoder.inverse_transform(y_pred)

print(classification_report(y_test, y_pred_decoded))


              precision    recall  f1-score   support

       blues       0.88      0.89      0.89       253
   classical       0.95      0.95      0.95       260
     country       0.88      0.93      0.90       249
       disco       0.90      0.90      0.90       250
      hiphop       0.95      0.88      0.91       235
        jazz       0.90      0.95      0.92       245
       metal       0.95      0.95      0.95       261
         pop       0.92      0.92      0.92       235
      reggae       0.91      0.89      0.90       258
        rock       0.87      0.84      0.85       250

    accuracy                           0.91      2496
   macro avg       0.91      0.91      0.91      2496
weighted avg       0.91      0.91      0.91      2496



In [7]:
# CatBoost model
y = df_new["label"].values
X = df_new.drop(columns='label').values

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

catboost_model = CatBoostClassifier(
    iterations=1000,       
    learning_rate=0.1,    
    depth=6,                  
    loss_function='MultiClass',
    random_seed=42,     
    verbose=50        
)

catboost_model.fit(X_train, y_train)
y_pred = catboost_model.predict(X_test)
y_pred_decoded = label_encoder.inverse_transform(y_pred)
y_test_decoded = label_encoder.inverse_transform(y_test)
print(classification_report(y_test_decoded, y_pred_decoded))

0:	learn: 2.1412984	total: 242ms	remaining: 4m 1s
50:	learn: 0.8119164	total: 1.56s	remaining: 29.1s
100:	learn: 0.5721732	total: 2.92s	remaining: 26s
150:	learn: 0.4301272	total: 4.17s	remaining: 23.5s
200:	learn: 0.3534094	total: 5.44s	remaining: 21.6s
250:	learn: 0.2951511	total: 6.75s	remaining: 20.1s
300:	learn: 0.2533230	total: 8.28s	remaining: 19.2s
350:	learn: 0.2175887	total: 9.69s	remaining: 17.9s
400:	learn: 0.1922190	total: 11.3s	remaining: 16.9s
450:	learn: 0.1686257	total: 12.9s	remaining: 15.7s
500:	learn: 0.1511684	total: 14.2s	remaining: 14.1s
550:	learn: 0.1358550	total: 15.4s	remaining: 12.6s
600:	learn: 0.1227655	total: 16.7s	remaining: 11.1s
650:	learn: 0.1129167	total: 17.9s	remaining: 9.61s
700:	learn: 0.1029099	total: 19.3s	remaining: 8.25s
750:	learn: 0.0939723	total: 20.7s	remaining: 6.88s
800:	learn: 0.0869031	total: 22s	remaining: 5.46s
850:	learn: 0.0799880	total: 23.2s	remaining: 4.06s
900:	learn: 0.0733381	total: 24.5s	remaining: 2.69s
950:	learn: 0.06869

C:\Users\husey\anaconda3\envs\dev\Lib\site-packages\sklearn\preprocessing\_label.py:151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [8]:
# Saving the CatBoost model
# catboost_model.save_model('model/catboost_model_with_label_encoding.cb')
# with open('model/label_encoder.pkl', 'wb') as f:
#     pickle.dump(label_encoder, f)